# NEXUS tool: case study for the Jordan - energy demand calculations
In this notebook a case study for the Jordan country is covered using the `nexus_tool` package. The water requirements for agricultural irrigation, residential, industrial and tourism use were previously calculated using the Water Evaluation and Planning System (WEAP) model. In this case study, the energy requirements for groundwater pumping, wastewater treatment, desalination of seawater and pumping for water conveyance are estimated.

First import the package by running the following block:

In [12]:
import sys
sys.path.append("..") #this is to add the avobe folder to the package directory
import os
import nexus_tool
import pandas as pd

## Read scenario data

In [13]:
data_folder = os.path.join('Data', 'Processed results')
scenario = 'Reference'
climate = 'Climate Change'
level = 'level_1'
input_folder = os.path.join(data_folder, scenario, climate, level)
results_folder = os.path.join('..','Jordan dashboard', 'data_test', scenario, climate, level)
os.makedirs(results_folder, exist_ok=True)
file_path = os.path.join(input_folder, 'pipelines_flow.csv')

## Create nexus model 

In [14]:
df = nexus_tool.read_csv(file_path)
jordan = nexus_tool.Model(df)

## Define pipelines diameters

In [15]:
jordan.df['Pipe_diameter'] = 1 #in m
jordan.df.loc[(df['pipeline']=='KAC'), 'Pipe_diameter'] = 4 #in m
jordan.df.loc[(df['pipeline']=='PL_Disi'), 'Pipe_diameter'] = 1.4 #in m
jordan.df.loc[(df['pipeline']=='PL_RedDead'), 'Pipe_diameter'] = 1.4 #in m
jordan.df.loc[(df['pipeline']=='PS_ZaraMain'), 'Pipe_diameter'] = 1.8 #in m
jordan.df.loc[(df['pipeline']=='PL_KACToZay'), 'Pipe_diameter'] = 1.4 #in m
jordan.df.loc[(df['pipeline']=='PL_Dab_AinGhazal'), 'Pipe_diameter'] = 1.4 #in m
jordan.df.loc[(df['pipeline']=='PL_ZaytoDabouq'), 'Pipe_diameter'] = 1.4 #in m

## Define variable names

In [16]:
jordan.elevation = 'elevation_delta'
jordan.L = 'segment_length'
jordan.peak_Q = 'value'
jordan.avg_Q = 'value'
jordan.SWpump_eff = 1 # pumping efficiency

## Calculate pumping energy requirements

In [17]:
jordan.get_A(inplace=True)
jordan.get_V(inplace=True, axis=0)
jordan.get_Re(inplace=True, axis=0)
jordan.get_f(inplace=True, axis=0)

jordan.get_sw_tdh(inplace = True, axis=0)
jordan.get_SWpumping_energy(inplace = True, axis=0)

## Create nexus model for groundwater pumping

In [18]:
# jordan.df['Q_per_meter_seg'] = jordan.df.value * jordan.df.segment_length / jordan.df.pipeline_length
# _vec = jordan.df.groupby(['Year','Month','pipeline'])['Q_per_meter_seg'].sum()
# jordan.df['Q_per_meter_pl'] = jordan.df.set_index(['Year', 'Month', 'pipeline']).index.map(_vec)

In [19]:
file_path = os.path.join(input_folder, 'groundwater_supply.csv')
df_groundwater = pd.read_csv(file_path)

jordan_gw = nexus_tool.Model(df_groundwater)

jordan_gw.df['Pipe_diameter'] = 0.4
jordan_gw.elevation = 'wtd_m'
jordan_gw.L = 'wtd_m'
jordan_gw.peak_Q = 'value'
jordan_gw.avg_Q = 'value'

## Caluculate groundwater pumping energy

In [20]:
jordan_gw.get_A(inplace=True)
jordan_gw.get_V(inplace=True, axis=0)
jordan_gw.get_Re(inplace=True, axis=0)
jordan_gw.get_f(inplace=True, axis=0)

jordan_gw.get_sw_tdh(inplace = True, axis=0)
jordan_gw.get_SWpumping_energy(inplace = True, axis=0)

## Calculate wastewater treatment energy

In [21]:
file_path = os.path.join(input_folder, 'wwtp_inflow.csv')
df_wwtp = pd.read_csv(file_path)

wwtp_energy_int = 0.6 # kWh/m3
df_wwtp['SWPA_E_'] = df_wwtp.value * wwtp_energy_int

## Calculate desalination energy

In [22]:
file_path = os.path.join(input_folder, 'Desalination.csv')
df_desal = pd.read_csv(file_path)

red_dead_energy_int = 3.31 # kWh/m3
aqaba_energy_int = 5 # kWh/m3
df_desal['SWPA_E_'] = 0

df_desal.loc[df_desal.point=='RedDead', 'SWPA_E_'] = df_desal.loc[df_desal.point=='RedDead', 'value'] * red_dead_energy_int
df_desal.loc[df_desal.point=='Aqaba Desal', 'SWPA_E_'] = df_desal.loc[df_desal.point=='Aqaba Desal', 'value'] * aqaba_energy_int

## Save result files

In [24]:
jordan.df.to_csv(os.path.join(results_folder, 'pipelines_data.csv'), index=False)
jordan_gw.df.to_csv(os.path.join(results_folder, 'groundwater_pumping.csv'), index=False)
df_wwtp.to_csv(os.path.join(results_folder, 'wwtp_data.csv'), index=False)
df_desal.to_csv(os.path.join(results_folder, 'desal_data.csv'), index=False)